## 2. Imports <a class='anchor' id='import'></a>

In [2]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

## 3 Load Data<a class='anchor' id='Load_Data'></a>

In [29]:
df_exc = pd.read_csv('C:\\Users\\somfl\\Documents\\Data Science Career Track\\LendingClub\\Exclusive.csv')
df_exc.set_index('id', inplace=True)
df_exc.head().T

id,68407277,68355089,68476807,68426831,68476668
revol_util,29.7,19.2,64.5,68.4,84.5
dti,5.91,16.06,25.37,10.2,14.67
chargeoff_within_12_mths,0.0,0.0,0.0,0.0,0.0
collections_12_mths_ex_med,0.0,0.0,0.0,0.0,0.0
inq_last_6mths,1.0,4.0,3.0,0.0,0.0
open_acc,7.0,22.0,12.0,5.0,12.0
mort_acc,1.0,4.0,6.0,0.0,4.0
annual_inc,55000.0,65000.0,104433.0,34000.0,180000.0
sub_grade,C4,C1,F1,C3,B2
funded_amnt,3600.0,24700.0,10400.0,11950.0,20000.0


In [27]:
df_exc.columns

Index(['revol_util', 'dti', 'chargeoff_within_12_mths',
       'collections_12_mths_ex_med', 'inq_last_6mths', 'open_acc', 'mort_acc',
       'annual_inc', 'sub_grade', 'funded_amnt', 'loan_status', 'revol_bal',
       'fico_range_high', 'fico_range_low', 'emp_length', 'home_ownership',
       'num_rev_accts', 'pub_rec_bankruptcies', 'tax_liens', 'Status',
       'Credit Length (year)'],
      dtype='object')

In [37]:
def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
    df = df.drop(columns='loan_status')
    y = df[y_col]
    X = df.drop(columns=y_col)
    X = pd.get_dummies(X, drop_first=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=42)  

    if std_cols:
        scaler = StandardScaler()
        X_train.loc[:, std_cols] = scaler.fit_transform(X_train[std_cols])
        X_test.loc[:, std_cols] = scaler.transform(X_test[std_cols])
    return X_train, X_test, y_train, y_test     

In [39]:
std_cols = ['revol_util', 'dti', 'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'inq_last_6mths', 'open_acc', 
'mort_acc', 'annual_inc', 'funded_amnt', 'revol_bal','fico_range_high', 'fico_range_low','num_rev_accts', 
'pub_rec_bankruptcies', 'tax_liens', 'Credit Length (year)']
X_train, X_test, y_train, y_test = get_train_test_X_y(df_exc, "Status", size=0.3, std_cols=std_cols)

C:\Users\somfl\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\somfl\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


## 4. Extract the Status of the loan <a class='anchor' id='extract'></a>

In [21]:
Y = df_exc['Status']
X = df_exc.drop(columns=['Status'])

In [22]:
X = pd.get_dummies(X, drop_first=True)

In [23]:
X.shape

(1206759, 70)

## 5. Split the data <a class='anchor' id='split'></a>

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=47)

In [25]:
X_train.dtypes

revol_util                    float64
dti                           float64
chargeoff_within_12_mths      float64
collections_12_mths_ex_med    float64
inq_last_6mths                float64
                               ...   
home_ownership_MORTGAGE         uint8
home_ownership_NONE             uint8
home_ownership_OTHER            uint8
home_ownership_OWN              uint8
home_ownership_RENT             uint8
Length: 70, dtype: object

## 6.Scale the data <a class='anchor' id='Scale'></a>

MACHINE LEARNING MODELS
This is a supervised machine learning regression analysis. Regression is used to predict continuous values. In this case, we're looking to predict the proportion or number of loans that will be defaulted. The following models were applied:

Linear Regression
Sklearn and Statsmodel
Detect multicollinearity with VIF (Variance Inflation Factor)
LASSO

Dealing with multicollinearity
Decision Tree

Random Forest
Pipeline
Feature importance
Gradient Boost
Metrics used to assess model performance:
MAE
MSE
RMSE
R_squared